# LDA

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel
from sklearn.feature_extraction.text import CountVectorizer
import pyLDAvis
import pyLDAvis.gensim
import pickle
import os
from gensim import corpora

## Participant-based analysis

### 1.  Dataset: Create the BOW vector from the concatenated strings
### * with BOW

In [3]:
from gensim.models import LdaMulticore, CoherenceModel

# Load and concatenate the DataFrames
def load_and_prepare_data(file_list):
    dfs = [pd.read_json(file, orient='index') for file in file_list]
    df = pd.concat(dfs, axis=1).fillna('')
    df.columns = ['response1', 'response2', 'response3']
    return df

# Preprocess data
def preprocess(df):
    combined_texts = df.apply(lambda row: ' '.join(row), axis=1)
    texts = combined_texts.str.split().tolist()
    return texts, combined_texts

# Train LDA model
def train_lda(corpus, dictionary, num_topics=5, random_state=42):
    lda_model = LdaMulticore(
        corpus=corpus, 
        id2word=dictionary, 
        num_topics=num_topics, 
        passes=10, 
        workers=4, 
        random_state=random_state
    )
    return lda_model

# Compute coherence score
def compute_coherence(lda_model, texts, dictionary):
    cm = CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='c_v')
    return cm.get_coherence()

In [4]:
# File list
files = [
    '/Users/gytkd/Desktop/Backup-Thesis/data/processed_uscensus/political_mention1.jsonl',
    '/Users/gytkd/Desktop/Backup-Thesis/data/processed_uscensus/political_mention2.jsonl',
    '/Users/gytkd/Desktop/Backup-Thesis/data/processed_uscensus/political_mention3.jsonl'
]

In [5]:
# Main process
df = load_and_prepare_data(files)
texts, combined_texts = preprocess(df)
dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [6]:
final_df = df.copy()

index_save = final_df.index 

In [7]:
# Loop to determine the best number of topics
num_topics_range = range(2, 11)  # Example range of topic numbers to try
coherence_scores = []
random_state = 0


### LDA model run below: untoggle the cell. 

In [8]:
# for num_topics in num_topics_range:
#     lda_model = train_lda(corpus, dictionary, num_topics=num_topics, random_state=random_state)
#     coherence_lda = compute_coherence(lda_model, texts, dictionary)
#     coherence_scores.append((num_topics, coherence_lda))
#     lda_model.save(f'lda_multicore_model_{num_topics}')  # Save model per topic number

In [9]:
# # Print coherence scores for each number of topics
# for num_topics, coherence_lda in coherence_scores:
#     print(f'Num Topics: {num_topics}, Coherence Score: {coherence_lda}')

# # Find the best number of topics
# best_num_topics, best_coherence_lda = max(coherence_scores, key=lambda item: item[1])
# print(f'Best Num Topics: {best_num_topics}, Best Coherence Score: {best_coherence_lda}')

### reload the LDA model

In [10]:
best_num_topics = 3

In [11]:
# Reload the best model
best_model_filepath = f'lda_multicore_model_{best_num_topics}'
best_lda_model = LdaMulticore.load(best_model_filepath)

In [12]:
# Step 9: Visualize the LDA Model using PyLDAvis
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('/Users/gytkd/Desktop/Backup-Thesis/lda-figure/anes/ldavis_' + str(best_num_topics))

# Prepare the visualization
if not os.path.exists(LDAvis_data_filepath):
    LDAvis_prepared = pyLDAvis.gensim.prepare(best_lda_model, corpus, dictionary)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
else:
    with open(LDAvis_data_filepath, 'rb') as f:
        LDAvis_prepared = pickle.load(f)

# Save the visualization as an HTML file
html_filepath = '/Users/gytkd/Desktop/Backup-Thesis/lda-figure/anes/ldavis_' + str(best_num_topics) + '.html'
pyLDAvis.save_html(LDAvis_prepared, html_filepath)

# Display the visualization inline (in Jupyter Notebook)
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.054600  0.048871       1        1  38.854301
2     -0.036763 -0.055675       2        1  36.190091
0      0.091363  0.006804       3        1  24.955607, topic_info=           Term        Freq        Total Category  logprob  loglift
13       health  700.000000   700.000000  Default  30.0000  30.0000
16       racism  932.000000   932.000000  Default  29.0000  29.0000
126     climate  565.000000   565.000000  Default  28.0000  28.0000
56         need  864.000000   864.000000  Default  27.0000  27.0000
125      change  530.000000   530.000000  Default  26.0000  26.0000
..          ...         ...          ...      ...      ...      ...
26       people  164.529005  2117.982302   Topic3  -5.1927  -1.1671
69   government  125.018796   587.914666   Topic3  -5.4673  -0.1600
332       media  116.299486   535.881253   Topic3  -5.5396  -0.1397
19           19  124.861247  1048.047839   Topic3  -5.4686  -0.7394
46      control  109.701183   521.239303   Topic3  -5.5981  -0.1704

[276 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
19        1  0.162206        19
19        2  0.718479        19
19        3  0.119269        19
67        1  0.787320      able
67        2  0.136469      able
...     ...       ...       ...
1491      1  0.923249  worrying
693       1  0.755273     would
693       2  0.157349     would
693       3  0.088115     would
5552      1  0.898378      zero

[519 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1])

## Connect the demographic information to the LDA results

In [13]:
# extract topic distribution for each document
topic_distributions = [best_lda_model.get_document_topics(doc, minimum_probability=0) for doc in corpus]
topic_df = pd.DataFrame([{i: prob for i, prob in doc} for doc in topic_distributions]).fillna(0)

In [14]:
# Assign topics to each document
most_probable_topics =  [max(topic, key=lambda x: x[1])[0] for topic in topic_distributions]

In [15]:
community= pd.DataFrame(most_probable_topics, index = index_save, columns = ['community'])

In [16]:
community

,community
200015,0
200022,1
200039,0
200046,2
200053,0
...,...
535414,1
535421,2
535469,1
200411,0


## Demographic dataset

In [42]:
# get the demographcis dataset
demos = pd.read_csv('/Users/gytkd/Desktop/Backup-Thesis/data/processed_data/anes_demographics.csv')

In [43]:
demos.set_index('id_case', inplace=True)

In [44]:
demos

,POST_vote,POST_president,PRE_present_religion,PRE_age,PRE_education,PRE_race,PRE_sex,PRE_occupation
id_case,,,,,,,,
200015,-1,-1,11,46,4,3,1,1
200022,1,3,12,37,3,4,2,1
200039,1,1,11,40,2,1,2,7
200046,1,1,2,41,3,4,1,1
200053,1,2,12,72,5,5,1,5
...,...,...,...,...,...,...,...,...
535315,-1,-1,11,26,3,1,2,1
535360,1,2,4,52,4,1,2,1
535414,1,1,2,45,2,3,1,1


### filter only the YES vote to the president

In [45]:
demos = demos[demos['POST_vote'] == 1]

In [46]:
demos

,POST_vote,POST_president,PRE_present_religion,PRE_age,PRE_education,PRE_race,PRE_sex,PRE_occupation
id_case,,,,,,,,
200022,1,3,12,37,3,4,2,1
200039,1,1,11,40,2,1,2,7
200046,1,1,2,41,3,4,1,1
200053,1,2,12,72,5,5,1,5
200060,1,1,10,71,3,1,2,5
...,...,...,...,...,...,...,...,...
535292,1,2,11,65,2,1,2,6
535308,1,2,11,54,4,1,2,1
535360,1,2,4,52,4,1,2,1


In [47]:
# Check for duplicate indices 
duplicates = demos.index.duplicated() # no duplicates exist

In [48]:
# Participants_ID converting from integer(index) to the original id from saved index.
final_df.index = index_save 
topic_df.index = index_save

### filter only the indices exist in demos for the clustering

In [49]:
final_df = final_df.loc[final_df.index.isin(demos.index)]
topic_df = topic_df.loc[topic_df.index.isin(demos.index)]
community = community.loc[community.index.isin(demos.index)]

In [50]:
community

,community
200022,1
200039,0
200046,2
200053,0
200060,1
...,...
535292,2
535308,0
535360,1
535414,1


### Make two different dataframe for the analysis: Topic == community 
* merged_df for the topic distributions
* community_df for the topic assignment

In [51]:
# make a common column for the merge
topic_df['id'] = topic_df.index
community['id'] = community.index
demos['id'] = demos.index

# merged_df: for the topic distributions
merged_df = pd.merge(topic_df, demos, on = 'id')
# community_df: for the community label. Here topic == community
community_df = pd.merge(community, demos, on = 'id')

/var/folders/j0/9xtsbl9502ng_j6cfbbp6fq00000gn/T/ipykernel_13936/912597800.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  community['id'] = community.index
/var/folders/j0/9xtsbl9502ng_j6cfbbp6fq00000gn/T/ipykernel_13936/912597800.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demos['id'] = demos.index


In [52]:
community_df.index = community_df['id']

In [53]:
community_df

,community,id,POST_vote,POST_president,PRE_present_religion,PRE_age,PRE_education,PRE_race,PRE_sex,PRE_occupation
id,,,,,,,,,,
200022,1,200022,1,3,12,37,3,4,2,1
200039,0,200039,1,1,11,40,2,1,2,7
200046,2,200046,1,1,2,41,3,4,1,1
200053,0,200053,1,2,12,72,5,5,1,5
200060,1,200060,1,1,10,71,3,1,2,5
...,...,...,...,...,...,...,...,...,...,...
535292,2,535292,1,2,11,65,2,1,2,6
535308,0,535308,1,2,11,54,4,1,2,1
535360,1,535360,1,2,4,52,4,1,2,1


In [54]:
# column name changed for the readability 
merged_df = merged_df.rename(columns = {0: 'community0', 1: 'community1', 2: 'community2'})
merged_df.head()

,community0,community1,community2,id,POST_vote,POST_president,PRE_present_religion,PRE_age,PRE_education,PRE_race,PRE_sex,PRE_occupation
0,0.092491,0.797120,0.110389,200022,1,3,12,37,3,4,2,1
1,0.938097,0.030269,0.031634,200039,1,1,11,40,2,1,2,7
2,0.209531,0.037027,0.753442,200046,1,1,2,41,3,4,1,1
3,0.967910,0.014659,0.017432,200053,1,2,12,72,5,5,1,5
4,0.020936,0.539041,0.440024,200060,1,1,10,71,3,1,2,5


### Create the community dictionary

### Age

In [55]:
# Initialize an empty dictionary to store DataFrames
community_dfs_age = {}

# Loop over each community label and create a DataFrame for it
for label in range(best_num_topics):
    community_dfs_age[label] = community_df[community_df['community'] == label][['PRE_age']]

# Display the DataFrame for a specific community, e.g., community 0
print(f"DataFrame for community 0:\n{community_dfs_age[0]}")

DataFrame for community 0:
        PRE_age
id             
200039       40
200053       72
200084       37
200275       80
200282       24
...         ...
534640       37
534725       25
534831       80
535100       67
535308       54

[1778 rows x 1 columns]


In [56]:
community_dfs_age

{0:         PRE_age
 id             
 200039       40
 200053       72
 200084       37
 200275       80
 200282       24
 ...         ...
 534640       37
 534725       25
 534831       80
 535100       67
 535308       54
 
 [1778 rows x 1 columns],
 1:         PRE_age
 id             
 200022       37
 200060       71
 200329       73
 200374       50
 200404       41
 ...         ...
 534596       80
 535254       47
 535360       52
 535414       45
 535469       38
 
 [1709 rows x 1 columns],
 2:         PRE_age
 id             
 200046       41
 200114       43
 200121       37
 200138       55
 200152       30
 ...         ...
 534534       49
 534589       58
 534992       46
 535032       68
 535292       65
 
 [2378 rows x 1 columns]}

### Political leaning

In [57]:
# Initialize an empty dictionary to store DataFrames
community_dfs_politics = {}

# Loop over each community label and create a DataFrame for it
for label in range(best_num_topics):
    community_dfs_politics[label] = community_df[community_df['community'] == label][['POST_president']]

# Display the DataFrame for a specific community, e.g., community 0
print(f"DataFrame for community 0:\n{community_dfs_politics[0]}")

DataFrame for community 0:
        POST_president
id                    
200039               1
200053               2
200084               2
200275               1
200282               1
...                ...
534640               1
534725               1
534831               1
535100               1
535308               2

[1778 rows x 1 columns]


### Religion

In [58]:
# Initialize an empty dictionary to store DataFrames
community_dfs_religion = {}

# Loop over each community label and create a DataFrame for it
for label in range(best_num_topics):
    community_dfs_religion[label] = community_df[community_df['community'] == label][['PRE_present_religion']]

# Display the DataFrame for a specific community, e.g., community 0
print(f"DataFrame for community 0:\n{community_dfs_religion[0]}")

DataFrame for community 0:
        PRE_present_religion
id                          
200039                    11
200053                    12
200084                     1
200275                     2
200282                    10
...                      ...
534640                     9
534725                    12
534831                     1
535100                    12
535308                    11

[1778 rows x 1 columns]


### Education

In [59]:
# Initialize an empty dictionary to store DataFrames
community_dfs_education = {}

# Loop over each community label and create a DataFrame for it
for label in range(best_num_topics):
    community_dfs_education[label] = community_df[community_df['community'] == label][['PRE_education']]

# Display the DataFrame for a specific community, e.g., community 0
print(f"DataFrame for community 0:\n{community_dfs_education[0]}")

DataFrame for community 0:
        PRE_education
id                   
200039              2
200053              5
200084              3
200275              1
200282              4
...               ...
534640              3
534725              3
534831              5
535100              1
535308              4

[1778 rows x 1 columns]


### Race

In [60]:
# Initialize an empty dictionary to store DataFrames
community_dfs_race = {}

# Loop over each community label and create a DataFrame for it
for label in range(best_num_topics):
    community_dfs_race[label] = community_df[community_df['community'] == label][['PRE_race']]

# Display the DataFrame for a specific community, e.g., community 0
print(f"DataFrame for community 0:\n{community_dfs_race[0]}")

DataFrame for community 0:
        PRE_race
id              
200039         1
200053         5
200084         1
200275         3
200282         1
...          ...
534640         5
534725        -8
534831         1
535100         1
535308         1

[1778 rows x 1 columns]


### Sex

In [61]:
# Initialize an empty dictionary to store DataFrames
community_dfs_sex = {}

# Loop over each community label and create a DataFrame for it
for label in range(best_num_topics):
    community_dfs_sex[label] = community_df[community_df['community'] == label][['PRE_sex']]

# Display the DataFrame for a specific community, e.g., community 0
print(f"DataFrame for community 0:\n{community_dfs_sex[0]}")

DataFrame for community 0:
        PRE_sex
id             
200039        2
200053        1
200084        2
200275        2
200282        1
...         ...
534640        1
534725        1
534831        2
535100        1
535308        2

[1778 rows x 1 columns]


### Job

In [62]:
# Initialize an empty dictionary to store DataFrames
community_dfs_job = {}

# Loop over each community label and create a DataFrame for it
for label in range(best_num_topics):
    community_dfs_job[label] = community_df[community_df['community'] == label][['PRE_occupation']]

# Display the DataFrame for a specific community, e.g., community 0
print(f"DataFrame for community 0:\n{community_dfs_job[0]}")

DataFrame for community 0:
        PRE_occupation
id                    
200039               7
200053               5
200084               1
200275               5
200282               1
...                ...
534640               1
534725              -2
534831               2
535100               5
535308               1

[1778 rows x 1 columns]


## Statistical Analysis

In [64]:
%pip install statsmodels

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 6.0 MB/s eta 0:00:0036m0:00:01m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 6.4 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.3/30.3 MB 6.8 MB/s eta 0:00:00m eta 0:00:010:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.1
    Uninstalling numpy-1.22.1:
      Successfully uninstalled numpy-1.22.1
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling scipy-1.7.3:
      Successfully uninstalled scipy-1.7.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pymc 5.12.0 requires arviz>=0.13.0, but you have arviz 0.12.1 which is 

In [65]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats
import statsmodels.multivariate.manova as MANOVA

ImportError: cannot import name 'copy_if_needed' from 'scipy._lib._util' (/Users/gytkd/miniforge3/lib/python3.9/site-packages/scipy/_lib/_util.py)

## ANOVA: use the community label

### Age

In [ ]:
for label, df_age in community_dfs_age.items():
    print(f"*community_{label} in age")
    print("how many:", len(df_age))
    print("mean:", np.mean(df_age['PRE_age']))
    print("variance:", np.var(df_age['PRE_age']))
    print("std:", np.std(df_age['PRE_age']))
    print()

## Levene's Test

In [ ]:
import scipy.stats as stats

In [ ]:
# Extract age data from each community's DataFrame
age_groups = [community_dfs_age[label]['PRE_age'].values for label in range(best_num_topics)]

# Perform Levene's test for homogeneity of variances
statistic, p_value = stats.levene(*age_groups)

# Print the results
print("Levene's Test Statistic:", statistic)
print("P-value:", p_value)

# Check the p-value against the significance level (commonly 0.05)
if p_value < 0.05:
    print("Reject the null hypothesis. There is evidence of unequal variances.")
else:
    print("Fail to reject the null hypothesis. Variances are likely homogeneous.")

## ANOVA

In [ ]:
# Perform one-way ANOVA
f_statistic, p_value = stats.f_oneway(*age_groups)

# Print the results
print("ANOVA F-Statistic:", f_statistic)
print("P-value:", p_value)

# Check the p-value against the significance level (commonly 0.05)
if p_value < 0.05:
    print("Reject the null hypothesis. There is evidence of significant differences in means.")
else:
    print("Fail to reject the null hypothesis. Means are likely equal across groups.")

**Result**: It means, there will be at least one population mean that differs from the rest, and it is not guaranteed that every population group(community) has different mean. 

## Follow-up Analysis on the ANOVA

In [ ]:
community_df.head()

In [ ]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

tukey = pairwise_tukeyhsd(endog=community_df['PRE_age'],
                          groups=community_df['community'],
                          alpha=0.05)

In [ ]:
print(tukey)

**Result**: 'reject': True means that the means of the two groups are equal is rejected. 
'reject': False means that the means of the two groups are equal is not rejected. 

### Post-hoc analysis: standardized residuals

In [ ]:
df_age = community_df[['PRE_age', 'community']]

In [ ]:
df_age

In [ ]:
# make the dataframe only with the age. 

# Calculate the group means
group_means = df_age.groupby('community')['PRE_age'].mean()

# Calculate residuals for each observation
df_age['residual'] = df_age.apply(lambda row: row['PRE_age'] - group_means[row['community']], axis=1)

# Calculate the standard deviation of the residuals
residual_std = np.std(df_age['residual'], ddof=1)

# Calculate standardized residuals
df_age['standardized_residual'] = df_age['residual'] / residual_std

# Display standardized residuals by group
print("\nStandardized Residuals by Group:")
for group, data in df_age.groupby('community'):
    print(f"\nGroup {group}:")
    print(data[['PRE_age', 'residual', 'standardized_residual']].head())  # Show top few rows for each group

# Summary statistics of standardized residuals by group
print("\nSummary of Standardized Residuals by Group:")
print(df_age.groupby('community')['standardized_residual'].describe())

**Interpretation on the standardized residuals**
* mean: all communities' are centered on 0 as it is standardized. 
* standard deviations(std): Groups with higher value suggests some observations in these groups are more atypical compared to others. Here, the deviations are alike between communities. 
* min and max: The ranges are bigger than expected, it means there are significant variations exist. 

## Chi-Square Test

### First, check the expected frequencies
* It is supposed to be >= 5 every cell in contingency_table.

### Political leaning

In [ ]:
combined_df = pd.concat([
    df.assign(community_label = label)
    for label, df in community_dfs_politics.items()
])

In [ ]:
combined_df.head()

In [ ]:
# Create a contingency table
contingency_table = pd.crosstab(combined_df['POST_president'], combined_df['community_label'])

# Show the contingency table
print(contingency_table) # it shows the change by the cell. 

### Cut off the dataset

In [ ]:
contingency_table.index

In [ ]:
# Retain only rows with index 1 and 2
contingency_table_filtered = contingency_table.loc[[1, 2]]

In [ ]:
# this is the table with only the people who voted for the Joe Biden/Donald Trump. 
contingency_table_filtered

In [ ]:
# Perform Chi-Square test
chi2_statistic, p_value, dof, expected = stats.chi2_contingency(contingency_table_filtered)

# Print the results
print("Expected Frequencies:")
print(expected)

# Check if all expected frequencies are >= 5
if np.all(expected >= 5):
    print("All expected frequencies are >= 5. The Chi-Square test can be used.")
else:
    print("Some expected frequencies are < 5. Consider using Fisher's Exact Test or combining categories.")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create a heatmap for the contingency table
sns.heatmap(contingency_table_filtered, annot=True, fmt='d')
plt.title("Contingency Table Heatmap")
plt.show()

### Chi-Square test

In [ ]:
# Perform the Chi-squared test
chi2, p, dof, expected = stats.chi2_contingency(contingency_table_filtered)

# Print the results in a readable format
print(f"{'Chi-Squared Test Results':^40}")
print("="*40)
print(f"{'Test Statistic (Chi2):':<25} {chi2:.4f}")
print(f"{'Degrees of Freedom:':<25} {dof}")
print(f"{'p-value:':<25} {p:.4e}")
print("\nExpected Frequencies (rounded):")
print(pd.DataFrame(expected, 
                   index=contingency_table_filtered.index, 
                   columns=contingency_table_filtered.columns).round(2))

### Influence of a label on the chi-square test: residuals calculations

In [ ]:
# Calculate the residuals
residuals = contingency_table_filtered - expected

# Standardize the residuals
standardized_residuals = residuals / np.sqrt(expected)

# Convert to DataFrame for better readability
standardized_residuals_df = pd.DataFrame(standardized_residuals,
                                         index=contingency_table_filtered.index,
                                         columns=contingency_table_filtered.columns)

# Print results
print(f"{'Chi-Squared Test Results':^40}")
print("="*40)
print(f"{'Test Statistic (Chi2):':<25} {chi2:.4f}")
print(f"{'Degrees of Freedom:':<25} {dof}")
print(f"{'p-value:':<25} {p:.4e}")
print("\nExpected Frequencies (rounded):")
print(pd.DataFrame(expected, 
                   index=contingency_table_filtered.index, 
                   columns=contingency_table_filtered.columns).round(2))
print("\nStandardized Residuals:")
print(standardized_residuals_df.round(2))

# Optional: if you want to see which residuals are extreme
print("\nExtreme Standardized Residuals (|value| > 2):")
print(standardized_residuals_df[standardized_residuals_df.abs() > 2].dropna(how='all'))


**Result:**
* The distribution of the feature(**political leaning**) **is significantly different** across different clusters. 
* Extreme Standard residuals exist in all cells except community 1. It means that substantial deviations from expected counts in specific community-label combinations. It highlights where the actual distribution of 'POST_president' significantly differs from what was expected under the null hypothesis. In other words, the variable 'POST_president' is distributed unevenly across the communities, but not in Community 1.  

### Pairwise Comparison with residuals interpretation

In [ ]:
import itertools
from statsmodels.stats.multitest import multipletests

In [ ]:
# Get all unique pairs of columns (communities)
pairs = list(itertools.combinations(contingency_table_filtered.columns, 2))

# Store results for pairwise comparisons
pairwise_results = []
pairwise_residuals = {}

In [ ]:
# Perform pairwise chi-squared tests
for pair in pairs:
    # Create a contingency table for each pair of communities
    pair_table = contingency_table_filtered.loc[:, pair]
    
    # Perform chi-squared test
    chi2_pair, p_pair, dof_pair, expected = stats.chi2_contingency(pair_table)
    
    # Calculate residuals and standardized residuals
    residuals = pair_table - expected
    standardized_residuals = residuals / np.sqrt(expected)
    
    # Append results
    pairwise_results.append((pair, chi2_pair, p_pair))
    pairwise_residuals[pair] = standardized_residuals

In [ ]:
# Extract p-values for correction
p_values = [result[2] for result in pairwise_results]

In [ ]:
# Apply Bonferroni correction
adjusted_p_values = multipletests(p_values, method='bonferroni')[1]

In [ ]:
# Print pairwise results with adjusted p-values
print("\nPairwise Chi-Squared Test Results (with Bonferroni correction):")
print("="*65)
print(f"{'Community Pair':<25}{'Chi2':<10}{'p-value':<15}{'Adj. p-value':<15}")
print("-"*65)

for i, (pair, chi2_pair, p_pair) in enumerate(pairwise_results):
    print(f"{str(pair):<25}{chi2_pair:<10.4f}{p_pair:<15.4e}{adjusted_p_values[i]:<15.4e}")

* Result: Comparing pairwise, 
    * (0, 1) community: adjusted p-value is <<< 0.05 : signiciant difference
    * (0, 2) community: adjusted p-value is << 0.05 : signiciant difference
    * (1, 2) community: adjusted p-value << 0.05: significant difference 

In [ ]:
# Print standardized residuals for each pairwise comparison
print("\nStandardized Residuals for Each Pairwise Comparison:")
for pair, residuals_df in pairwise_residuals.items():
    print(f"\nPair: {pair}")
    print(pd.DataFrame(residuals_df, index=contingency_table_filtered.index, columns=pair).round(2))

**Interpretation: Residuals**
* Significant deviations from expected frequencies are observed in all pairs but especially the most on (0, 2).  

### Education

In [ ]:
combined_df = pd.concat([
    df.assign(community_label = label)
    for label, df in community_dfs_education.items()
])

In [ ]:
combined_df.head()

In [ ]:
# Create a contingency table
contingency_table = pd.crosstab(combined_df['PRE_education'], combined_df['community_label'])

# Show the contingency table
print(contingency_table) # it shows the change by the cell. 

In [ ]:
# this is the table with only the people who voted for the Joe Biden/Donald Trump. 
contingency_table

In [ ]:
contingency_table_filtered = contingency_table.loc[[-2, 1, 2, 3, 4, 5]]

In [ ]:
# Perform Chi-Square test
chi2_statistic, p_value, dof, expected = stats.chi2_contingency(contingency_table_filtered)

# Print the results
print("Expected Frequencies:")
print(expected)

# Check if all expected frequencies are >= 5
if np.all(expected >= 5):
    print("All expected frequencies are >= 5. The Chi-Square test can be used.")
else:
    print("Some expected frequencies are < 5. Consider using Fisher's Exact Test or combining categories.")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create a heatmap for the contingency table
sns.heatmap(contingency_table, annot=True, fmt='d')
plt.title("Contingency Table Heatmap")
plt.show()

### Chi-Square test

In [ ]:
# Perform the Chi-squared test
chi2, p, dof, expected = stats.chi2_contingency(contingency_table_filtered)

# Print the results in a readable format
print(f"{'Chi-Squared Test Results':^40}")
print("="*40)
print(f"{'Test Statistic (Chi2):':<25} {chi2:.4f}")
print(f"{'Degrees of Freedom:':<25} {dof}")
print(f"{'p-value:':<25} {p:.4e}")
print("\nExpected Frequencies (rounded):")
print(pd.DataFrame(expected, 
                   index=contingency_table_filtered.index, 
                   columns=contingency_table_filtered.columns).round(2))

**Result**: It says, the educationn shows the significant difference between the clusters

### Influence of a label on the chi-square test: residuals calculations

In [ ]:
# Calculate the residuals
residuals = contingency_table_filtered - expected

# bStandardize the residuals
standardized_residuals = residuals / np.sqrt(expected)

# Convert to DataFrame for better readability
standardized_residuals_df = pd.DataFrame(standardized_residuals,
                                         index=contingency_table_filtered.index,
                                         columns=contingency_table_filtered.columns)

# Print results
print(f"{'Chi-Squared Test Results':^40}")
print("="*40)
print(f"{'Test Statistic (Chi2):':<25} {chi2:.4f}")
print(f"{'Degrees of Freedom:':<25} {dof}")
print(f"{'p-value:':<25} {p:.4e}")
print("\nExpected Frequencies (rounded):")
print(pd.DataFrame(expected, 
                   index=contingency_table_filtered.index, 
                   columns=contingency_table_filtered.columns).round(2))
print("\nStandardized Residuals:")
print(standardized_residuals_df.round(2))

# Optional: if you want to see which residuals are extreme
print("\nExtreme Standardized Residuals (|value| > 2):")
print(standardized_residuals_df[standardized_residuals_df.abs() > 2].dropna(how='all'))


**Result:**
* The distribution of the feature(**education**) **is significantly different** across different clusters. 
* The extreme standardized residuals showed that community 0 is outstandingly different between the communities for its distributions regarding the labels. 

### Pairwise Comparison with residuals interpretation

In [ ]:
import itertools
from statsmodels.stats.multitest import multipletests

In [ ]:
# Get all unique pairs of columns (communities)
pairs = list(itertools.combinations(contingency_table_filtered.columns, 2))

# Store results for pairwise comparisons
pairwise_results = []
pairwise_residuals = {}

In [ ]:
# Perform pairwise chi-squared tests
for pair in pairs:
    # Create a contingency table for each pair of communities
    pair_table = contingency_table_filtered.loc[:, pair]
    
    # Perform chi-squared test
    chi2_pair, p_pair, dof_pair, expected = stats.chi2_contingency(pair_table)
    
    # Calculate residuals and standardized residuals
    residuals = pair_table - expected
    standardized_residuals = residuals / np.sqrt(expected)
    
    # Append results
    pairwise_results.append((pair, chi2_pair, p_pair))
    pairwise_residuals[pair] = standardized_residuals

In [ ]:
# Extract p-values for correction
p_values = [result[2] for result in pairwise_results]

In [ ]:
# Apply Bonferroni correction
adjusted_p_values = multipletests(p_values, method='bonferroni')[1]

In [ ]:
# Print pairwise results with adjusted p-values
print("\nPairwise Chi-Squared Test Results (with Bonferroni correction):")
print("="*65)
print(f"{'Community Pair':<25}{'Chi2':<10}{'p-value':<15}{'Adj. p-value':<15}")
print("-"*65)

for i, (pair, chi2_pair, p_pair) in enumerate(pairwise_results):
    print(f"{str(pair):<25}{chi2_pair:<10.4f}{p_pair:<15.4e}{adjusted_p_values[i]:<15.4e}")

* Result: Comparing pairwise, 
    * (0, 1) community: adjusted p-value is << 0.05 : signiciant difference
    * (0, 2) community: adjusted p-value is << 0.05 : signiciant difference
    * (1, 2) community: adjusted p-value is > 0.05 : no signiciant difference

In [ ]:
# Print standardized residuals for each pairwise comparison
print("\nStandardized Residuals for Each Pairwise Comparison:")
for pair, residuals_df in pairwise_residuals.items():
    print(f"\nPair: {pair}")
    print(pd.DataFrame(residuals_df, index=contingency_table_filtered.index, columns=pair).round(2))

**Interpretation: Residuals**
* compared to the pair (1, 2), the other pairs show more discrepancies in their distributions as having bigger absolute values of the standardized residuals when comparing between the 2 community. 

### Sex

In [ ]:
combined_df = pd.concat([
    df.assign(community_label = label)
    for label, df in community_dfs_sex.items()
])

In [ ]:
combined_df.head()

In [ ]:
# Create a contingency table
contingency_table = pd.crosstab(combined_df['PRE_sex'], combined_df['community_label'])

# Show the contingency table
print(contingency_table) # it shows the change by the cell. 

In [ ]:
contingency_table.index

In [ ]:
contingency_table_filtered = contingency_table.loc[[1, 2]]

In [ ]:
# Perform Chi-Square test
chi2_statistic, p_value, dof, expected = stats.chi2_contingency(contingency_table_filtered)

# Print the results
print("Expected Frequencies:")
print(expected)

# Check if all expected frequencies are >= 5
if np.all(expected >= 5):
    print("All expected frequencies are >= 5. The Chi-Square test can be used.")
else:
    print("Some expected frequencies are < 5. Consider using Fisher's Exact Test or combining categories.")

### Chi-Square test

In [ ]:
# Perform the Chi-squared test
chi2, p, dof, expected = stats.chi2_contingency(contingency_table_filtered)

# Print the results in a readable format
print(f"{'Chi-Squared Test Results':^40}")
print("="*40)
print(f"{'Test Statistic (Chi2):':<25} {chi2:.4f}")
print(f"{'Degrees of Freedom:':<25} {dof}")
print(f"{'p-value:':<25} {p:.4e}")
print("\nExpected Frequencies (rounded):")
print(pd.DataFrame(expected, 
                   index=contingency_table_filtered.index, 
                   columns=contingency_table_filtered.columns).round(2))

**Result**: It says, the sex shows the significant difference between the clusters

### Influence of a label on the chi-square test: residuals calculations

In [ ]:
# Calculate the residuals
residuals = contingency_table_filtered - expected

# bStandardize the residuals
standardized_residuals = residuals / np.sqrt(expected)

# Convert to DataFrame for better readability
standardized_residuals_df = pd.DataFrame(standardized_residuals,
                                         index=contingency_table_filtered.index,
                                         columns=contingency_table_filtered.columns)

# Print results
print(f"{'Chi-Squared Test Results':^40}")
print("="*40)
print(f"{'Test Statistic (Chi2):':<25} {chi2:.4f}")
print(f"{'Degrees of Freedom:':<25} {dof}")
print(f"{'p-value:':<25} {p:.4e}")
print("\nExpected Frequencies (rounded):")
print(pd.DataFrame(expected, 
                   index=contingency_table_filtered.index, 
                   columns=contingency_table_filtered.columns).round(2))
print("\nStandardized Residuals:")
print(standardized_residuals_df.round(2))

# Optional: if you want to see which residuals are extreme
print("\nExtreme Standardized Residuals (|value| > 2):")
print(standardized_residuals_df[standardized_residuals_df.abs() > 2].dropna(how='all'))


**Result:**
* The distribution of the feature(**sex**) is **not significantly different** across different clusters. 
* The lack of extreme residuals tell that the difference of distributions of the variable is not influenced by a specific cell. 

### Pairwise Comparison with residuals interpretation

In [ ]:
import itertools
from statsmodels.stats.multitest import multipletests

In [ ]:
# Get all unique pairs of columns (communities)
pairs = list(itertools.combinations(contingency_table_filtered.columns, 2))

# Store results for pairwise comparisons
pairwise_results = []
pairwise_residuals = {}

In [ ]:
# Perform pairwise chi-squared tests
for pair in pairs:
    # Create a contingency table for each pair of communities
    pair_table = contingency_table_filtered.loc[:, pair]
    
    # Perform chi-squared test
    chi2_pair, p_pair, dof_pair, expected = stats.chi2_contingency(pair_table)
    
    # Calculate residuals and standardized residuals
    residuals = pair_table - expected
    standardized_residuals = residuals / np.sqrt(expected)
    
    # Append results
    pairwise_results.append((pair, chi2_pair, p_pair))
    pairwise_residuals[pair] = standardized_residuals

In [ ]:
# Extract p-values for correction
p_values = [result[2] for result in pairwise_results]

In [ ]:
# Apply Bonferroni correction
adjusted_p_values = multipletests(p_values, method='bonferroni')[1]

In [ ]:
# Print pairwise results with adjusted p-values
print("\nPairwise Chi-Squared Test Results (with Bonferroni correction):")
print("="*65)
print(f"{'Community Pair':<25}{'Chi2':<10}{'p-value':<15}{'Adj. p-value':<15}")
print("-"*65)

for i, (pair, chi2_pair, p_pair) in enumerate(pairwise_results):
    print(f"{str(pair):<25}{chi2_pair:<10.4f}{p_pair:<15.4e}{adjusted_p_values[i]:<15.4e}")

* Result: Comparing pairwise, 
    * (0, 1) community: adjusted p-value is > 0.05 : no signiciant difference
    * (0, 2) community: adjusted p-value is > 0.05 : no signiciant difference
    * (1, 2) community: adjusted p-value > 0.05: no significant difference

In [ ]:
# Print standardized residuals for each pairwise comparison
print("\nStandardized Residuals for Each Pairwise Comparison:")
for pair, residuals_df in pairwise_residuals.items():
    print(f"\nPair: {pair}")
    print(pd.DataFrame(residuals_df, index=contingency_table_filtered.index, columns=pair).round(2))

**Interpretation: Residuals**
* compared to the other pair, pair (1, 2) showed the most difference to each other. 

### Employment

In [ ]:
combined_df = pd.concat([
    df.assign(community_label = label)
    for label, df in community_dfs_job.items()
])

In [ ]:
combined_df.head()

In [ ]:
# Create a contingency table
contingency_table = pd.crosstab(combined_df['PRE_occupation'], combined_df['community_label'])

# Show the contingency table
print(contingency_table) # it shows the change by the cell. 

In [ ]:
# Perform Chi-Square test
chi2_statistic, p_value, dof, expected = stats.chi2_contingency(contingency_table)

# Print the results
print("Expected Frequencies:")
print(expected)

# Check if all expected frequencies are >= 5
if np.all(expected >= 5):
    print("All expected frequencies are >= 5. The Chi-Square test can be used.")
else:
    print("Some expected frequencies are < 5. Consider using Fisher's Exact Test or combining categories.")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create a heatmap for the contingency table
sns.heatmap(contingency_table, annot=True, fmt='d')
plt.title("Contingency Table Heatmap")
plt.show()

### Chi-Square test

In [ ]:
# Perform the Chi-squared test
chi2, p, dof, expected = stats.chi2_contingency(contingency_table)

# Print the results in a readable format
print(f"{'Chi-Squared Test Results':^40}")
print("="*40)
print(f"{'Test Statistic (Chi2):':<25} {chi2:.4f}")
print(f"{'Degrees of Freedom:':<25} {dof}")
print(f"{'p-value:':<25} {p:.4e}")
print("\nExpected Frequencies (rounded):")
print(pd.DataFrame(expected, 
                   index=contingency_table.index, 
                   columns=contingency_table.columns).round(2))

**Result**: It says, the Employment shows the significant difference between the clusters

### Influence of a label on the chi-square test: residuals calculations

In [ ]:
# Calculate the residuals
residuals = contingency_table - expected

# bStandardize the residuals
standardized_residuals = residuals / np.sqrt(expected)

# Convert to DataFrame for better readability
standardized_residuals_df = pd.DataFrame(standardized_residuals,
                                         index=contingency_table.index,
                                         columns=contingency_table.columns)

# Print results
print(f"{'Chi-Squared Test Results':^40}")
print("="*40)
print(f"{'Test Statistic (Chi2):':<25} {chi2:.4f}")
print(f"{'Degrees of Freedom:':<25} {dof}")
print(f"{'p-value:':<25} {p:.4e}")
print("\nExpected Frequencies (rounded):")
print(pd.DataFrame(expected, 
                   index=contingency_table.index, 
                   columns=contingency_table.columns).round(2))
print("\nStandardized Residuals:")
print(standardized_residuals_df.round(2))

# Optional: if you want to see which residuals are extreme
print("\nExtreme Standardized Residuals (|value| > 2):")
print(standardized_residuals_df[standardized_residuals_df.abs() > 2].dropna(how='all'))


**Result:**
* The distribution of the feature(**Jobs**) **is significantly different** across different clusters. 
* Residuals: needs to be filled

### Pairwise Comparison with residuals interpretation

In [ ]:
import itertools
from statsmodels.stats.multitest import multipletests

In [ ]:
# Get all unique pairs of columns (communities)
pairs = list(itertools.combinations(contingency_table.columns, 2))

# Store results for pairwise comparisons
pairwise_results = []
pairwise_residuals = {}

In [ ]:
# Perform pairwise chi-squared tests
for pair in pairs:
    # Create a contingency table for each pair of communities
    pair_table = contingency_table.loc[:, pair]
    
    # Perform chi-squared test
    chi2_pair, p_pair, dof_pair, expected = stats.chi2_contingency(pair_table)
    
    # Calculate residuals and standardized residuals
    residuals = pair_table - expected
    standardized_residuals = residuals / np.sqrt(expected)
    
    # Append results
    pairwise_results.append((pair, chi2_pair, p_pair))
    pairwise_residuals[pair] = standardized_residuals

In [ ]:
# Extract p-values for correction
p_values = [result[2] for result in pairwise_results]

In [ ]:
# Apply Bonferroni correction
adjusted_p_values = multipletests(p_values, method='bonferroni')[1]

In [ ]:
# Print pairwise results with adjusted p-values
print("\nPairwise Chi-Squared Test Results (with Bonferroni correction):")
print("="*65)
print(f"{'Community Pair':<25}{'Chi2':<10}{'p-value':<15}{'Adj. p-value':<15}")
print("-"*65)

for i, (pair, chi2_pair, p_pair) in enumerate(pairwise_results):
    print(f"{str(pair):<25}{chi2_pair:<10.4f}{p_pair:<15.4e}{adjusted_p_values[i]:<15.4e}")

* Result: Comparing pairwise, 
    * (0, 1) community: adjusted p-value is needs to be filled. 
    * (0, 2) community: adjusted p-value is needs to be filled. 
    * (1, 2) community: 

In [ ]:
# Print standardized residuals for each pairwise comparison
print("\nStandardized Residuals for Each Pairwise Comparison:")
for pair, residuals_df in pairwise_residuals.items():
    print(f"\nPair: {pair}")
    print(pd.DataFrame(residuals_df, index=contingency_table.index, columns=pair).round(2))

**Interpretation: Residuals**
* Extreme Residuals: needs to be filled
* Moderate Residuals: needs to be filled
* Small Residuals: needs to be filled

### Religion

In [ ]:
combined_df = pd.concat([
    df.assign(community_label = label)
    for label, df in community_dfs_religion.items()
])

In [ ]:
combined_df.head()

In [ ]:
# Create a contingency table
contingency_table = pd.crosstab(combined_df['PRE_present_religion'], combined_df['community_label'])

# Show the contingency table
print(contingency_table) # it shows the change by the cell. 

In [ ]:
# Perform Chi-Square test
chi2_statistic, p_value, dof, expected = stats.chi2_contingency(contingency_table)

# Print the results
print("Expected Frequencies:")
print(expected)

# Check if all expected frequencies are >= 5
if np.all(expected >= 5):
    print("All expected frequencies are >= 5. The Chi-Square test can be used.")
else:
    print("Some expected frequencies are < 5. Consider using Fisher's Exact Test or combining categories.")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create a heatmap for the contingency table
sns.heatmap(contingency_table, annot=True, fmt='d')
plt.title("Contingency Table Heatmap")
plt.show()

### Chi-Square test

In [ ]:
# Perform the Chi-squared test
chi2, p, dof, expected = stats.chi2_contingency(contingency_table)

# Print the results in a readable format
print(f"{'Chi-Squared Test Results':^40}")
print("="*40)
print(f"{'Test Statistic (Chi2):':<25} {chi2:.4f}")
print(f"{'Degrees of Freedom:':<25} {dof}")
print(f"{'p-value:':<25} {p:.4e}")
print("\nExpected Frequencies (rounded):")
print(pd.DataFrame(expected, 
                   index=contingency_table.index, 
                   columns=contingency_table.columns).round(2))

**Result**: It says, the Religion shows the significant difference between the clusters

### Influence of a label on the chi-square test: residuals calculations

In [ ]:
# Calculate the residuals
residuals = contingency_table - expected

# bStandardize the residuals
standardized_residuals = residuals / np.sqrt(expected)

# Convert to DataFrame for better readability
standardized_residuals_df = pd.DataFrame(standardized_residuals,
                                         index=contingency_table.index,
                                         columns=contingency_table.columns)

# Print results
print(f"{'Chi-Squared Test Results':^40}")
print("="*40)
print(f"{'Test Statistic (Chi2):':<25} {chi2:.4f}")
print(f"{'Degrees of Freedom:':<25} {dof}")
print(f"{'p-value:':<25} {p:.4e}")
print("\nExpected Frequencies (rounded):")
print(pd.DataFrame(expected, 
                   index=contingency_table.index, 
                   columns=contingency_table.columns).round(2))
print("\nStandardized Residuals:")
print(standardized_residuals_df.round(2))

# Optional: if you want to see which residuals are extreme
print("\nExtreme Standardized Residuals (|value| > 2):")
print(standardized_residuals_df[standardized_residuals_df.abs() > 2].dropna(how='all'))


**Result:**
* The distribution of the feature(**religion**) **is significantly different** across different clusters. 
* Residuals: needs to be filled

### Pairwise Comparison with residuals interpretation

In [ ]:
import itertools
from statsmodels.stats.multitest import multipletests

In [ ]:
# Get all unique pairs of columns (communities)
pairs = list(itertools.combinations(contingency_table.columns, 2))

# Store results for pairwise comparisons
pairwise_results = []
pairwise_residuals = {}

In [ ]:
# Perform pairwise chi-squared tests
for pair in pairs:
    # Create a contingency table for each pair of communities
    pair_table = contingency_table.loc[:, pair]
    
    # Perform chi-squared test
    chi2_pair, p_pair, dof_pair, expected = stats.chi2_contingency(pair_table)
    
    # Calculate residuals and standardized residuals
    residuals = pair_table - expected
    standardized_residuals = residuals / np.sqrt(expected)
    
    # Append results
    pairwise_results.append((pair, chi2_pair, p_pair))
    pairwise_residuals[pair] = standardized_residuals

In [ ]:
# Extract p-values for correction
p_values = [result[2] for result in pairwise_results]

In [ ]:
# Apply Bonferroni correction
adjusted_p_values = multipletests(p_values, method='bonferroni')[1]

In [ ]:
# Print pairwise results with adjusted p-values
print("\nPairwise Chi-Squared Test Results (with Bonferroni correction):")
print("="*65)
print(f"{'Community Pair':<25}{'Chi2':<10}{'p-value':<15}{'Adj. p-value':<15}")
print("-"*65)

for i, (pair, chi2_pair, p_pair) in enumerate(pairwise_results):
    print(f"{str(pair):<25}{chi2_pair:<10.4f}{p_pair:<15.4e}{adjusted_p_values[i]:<15.4e}")

* Result: Comparing pairwise, 
    * (0, 1) community: adjusted p-value is needs to be filled
    * (0, 2) community: adjusted p-value is needs to be filled
    * (1, 2) community: 

In [ ]:
# Print standardized residuals for each pairwise comparison
print("\nStandardized Residuals for Each Pairwise Comparison:")
for pair, residuals_df in pairwise_residuals.items():
    print(f"\nPair: {pair}")
    print(pd.DataFrame(residuals_df, index=contingency_table.index, columns=pair).round(2))

**Interpretation: Residuals**
* Extreme Residuals: needs to be filled
* Moderate Residuals: needs to be filled
* Small Residuals: needs to be filled

### 2.  Dataset: Create the tf-idf vector from the concatenated strings
### * with tf-idf

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
nltk.download('stopwords')
stop_words = stopwords.words('english')


In [ ]:
# Load and concatenate the DataFrames
def load_and_prepare_data(file_list):
    dfs = [pd.read_json(file, orient='index') for file in file_list]
    df = pd.concat(dfs, axis=1).fillna('')
    df.columns = ['response1', 'response2', 'response3']
    return df

# Preprocess data
def preprocess(df):
    return df.applymap(lambda x: ' '.join(simple_preprocess(str(x), deacc=True)))

# Tokenize and remove stopwords
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

In [ ]:
# Create TF-IDF vectors
def create_tfidf_vectors(df):
    vectorizer = TfidfVectorizer(max_features=1000)
    tfidf_matrix = vectorizer.fit_transform(df.values.flatten())
    feature_names = vectorizer.get_feature_names_out()
    return tfidf_matrix, feature_names, vectorizer

# Sum the vectors per document
def sum_vectors_per_document(tfidf_matrix, df):
    num_docs = df.shape[0]
    summed_vectors = np.zeros((num_docs, tfidf_matrix.shape[1]))

    for i in range(num_docs):
        summed_vectors[i] = tfidf_matrix[i*df.shape[1]:(i+1)*df.shape[1]].sum(axis=0)

    return summed_vectors

# Convert summed vectors to Gensim corpus
def convert_to_gensim_corpus(summed_vectors, feature_names):
    dictionary = Dictionary([feature_names.tolist()])
    corpus = []
    for summed_vec in summed_vectors:
        bow = [(i, summed_vec[i]) for i in range(len(summed_vec)) if summed_vec[i] > 0]
        corpus.append(bow)
    return corpus, dictionary


In [ ]:
# Train LDA model
def train_lda(corpus, dictionary, num_topics=5, random_state=42):
    lda_model = LdaMulticore(
        corpus=corpus, 
        id2word=dictionary, 
        num_topics=num_topics, 
        passes=10, 
        workers=4, 
        random_state=random_state
    )
    return lda_model

# Compute coherence score
def compute_coherence(lda_model, texts, dictionary):
    cm = CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='c_v')
    return cm.get_coherence()



In [ ]:
# File list
files = [
    '~/thesis/data/processed_uscensus/political_mention1.jsonl',
    '~/thesis/data/processed_uscensus/political_mention2.jsonl',
    '~/thesis/data/processed_uscensus/political_mention3.jsonl'
]


In [ ]:
# Main process
df = load_and_prepare_data(files)

# Pre-process and tokenize data responses
tokenized_responses = preprocess(df)
responses_flat_tokenized = tokenized_responses.values.flatten().tolist()
data_words = list(sent_to_words(responses_flat_tokenized))

In [ ]:
# Convert lists of words back into strings for TF-IDF vectorization
merged_data_words_strings = [' '.join(words) for words in data_words]

tfidf_matrix, feature_names, vectorizer = create_tfidf_vectors(tokenized_responses)
summed_vectors = sum_vectors_per_document(tfidf_matrix, tokenized_responses)
corpus, dictionary = convert_to_gensim_corpus(summed_vectors, feature_names)

In [ ]:
# Loop to determine the best number of topics
num_topics_range = range(2, 11)  # Example range of topic numbers to try
coherence_scores = []
random_state = 42


In [ ]:
for num_topics in num_topics_range:
    lda_model = train_lda(corpus, dictionary, num_topics=num_topics, random_state=random_state)
    coherence_lda = compute_coherence(lda_model, data_words, dictionary)
    coherence_scores.append((num_topics, coherence_lda))
    lda_model.save(f'lda_multicore_model_{num_topics}')  # Save model per topic number


In [ ]:
# Print coherence scores for each number of topics
for num_topics, coherence_lda in coherence_scores:
    print(f'Num Topics: {num_topics}, Coherence Score: {coherence_lda}')


In [ ]:
# Find the best number of topics
best_num_topics, best_coherence_lda = max(coherence_scores, key=lambda item: item[1])
print(f'Best Num Topics: {best_num_topics}, Best Coherence Score: {best_coherence_lda}')



In [ ]:
# Reload the best model
best_model_filepath = f'lda_multicore_model_{best_num_topics}'
best_lda_model = LdaMulticore.load(best_model_filepath)


In [ ]:
# Step 9: Visualize the LDA Model using PyLDAvis
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('/mnt/home/kim/thesis/lda-figure/anes/ldavis_participant_tfidf' + str(best_num_topics))

# Prepare the visualization
if not os.path.exists(LDAvis_data_filepath):
    LDAvis_prepared = pyLDAvis.gensim.prepare(best_lda_model, corpus, dictionary)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
else:
    with open(LDAvis_data_filepath, 'rb') as f:
        LDAvis_prepared = pickle.load(f)

# Save the visualization as an HTML file
html_filepath = '/mnt/home/kim/thesis/lda-figure/anes/ldavis_participant_tfidf' + str(best_num_topics) + '.html'
pyLDAvis.save_html(LDAvis_prepared, html_filepath)

# Display the visualization inline (in Jupyter Notebook)
LDAvis_prepared

## Linkage Matrix

In [ ]:
from sklearn.metrics.pairwise import cosine_distances
import scipy.cluster.hierarchy as sch
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [ ]:
# Data for the analysis - feature 1000 for each participant_id
final_df

### 1. Dimensionality reduction for the dendrogram(linkage matrix) visualization

### PCA (Principal Component Analysis)

In [ ]:
pca = PCA(n_components = 50)
pca_result = pca.fit_transform(final_df)

In [ ]:
distance_matrix = cosine_distances(pca_result)

In [ ]:
linkage_matrix = sch.linkage(distance_matrix, method = 'ward')

In [ ]:
# Step 4: Plot the Dendrogram
plt.figure(figsize=(10, 7))
sch.dendrogram(linkage_matrix, labels=final_df.index.tolist())
plt.title('Dendrogram for PCA-Reduced Data')
plt.xlabel('Documents')
plt.ylabel('Distance')
plt.show()

### UMAP

In [ ]:
import umap

In [ ]:
type(final_df)

In [ ]:
# change the data type to compatible one to UMAP
from sklearn.preprocessing import StandardScaler

# Example of standardizing data
scaler = StandardScaler()
final_df_scaled = scaler.fit_transform(final_df)
final_df_scaled = final_df_scaled.astype(float)

In [ ]:
# initialize the umap model
umap_model = umap.UMAP(n_components = 50, random_state = 42) 
umap_result = umap_model.fit_transform(final_df_scaled)

In [ ]:
distance_matrix = cosine_distances(pca_result)

In [ ]:
linkage_matrix = sch.linkage(distance_matrix, method = 'ward')

In [ ]:
# Step 4: Plot the Dendrogram
plt.figure(figsize=(10, 7))
sch.dendrogram(linkage_matrix, labels=final_df.index.tolist())
plt.title('Dendrogram for PCA-Reduced Data')
plt.xlabel('Documents')
plt.ylabel('Distance')
plt.show()

### 2. Heatmap instead of the Dendrogram

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_distances
import scipy.cluster.hierarchy as sch
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Convert DataFrame to numpy array if needed
if isinstance(final_df, pd.DataFrame):
    final_df = final_df.values

# Standardize data
scaler = StandardScaler()
final_df_scaled = scaler.fit_transform(final_df)

In [ ]:
# Apply PCA
pca = PCA(n_components=5)  # Reduce to 50 components or adjust as needed
pca_result = pca.fit_transform(final_df_scaled)

In [ ]:
# Compute the Distance Matrix
distance_matrix = cosine_distances(pca_result)

# Perform Hierarchical Clustering
linkage_matrix = sch.linkage(distance_matrix, method='ward')


In [ ]:
# Create a Cluster Dendrogram
dendro = sch.dendrogram(linkage_matrix, no_plot=True)
dendro_order = dendro['leaves']


In [ ]:
# Reorder the distance matrix
distance_matrix_reordered = distance_matrix[np.ix_(dendro_order, dendro_order)]

In [ ]:
# Plot the Heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(distance_matrix_reordered, cmap='viridis', cbar=True, 
            xticklabels=np.array(dendro_order)+1, yticklabels=np.array(dendro_order)+1)
plt.title('Heatmap of Clustering Distance Matrix (PCA)')
plt.xlabel('Documents')
plt.ylabel('Documents')
plt.show()


### 2. Dataset: responses stacked
### * with BOW

In [ ]:
import pandas as pd
import gensim
from gensim.utils import simple_preprocess
import nltk
from nltk.corpus import stopwords
import gensim.corpora as corpora
from gensim.models import LdaMulticore, CoherenceModel
import pyLDAvis.gensim
import pickle
import pyLDAvis
import os

In [ ]:
# Load data
data = pd.read_json('~/thesis/data/processed_uscensus/political_mentions_stack.jsonl', orient='records', lines=True)

In [ ]:
nltk.download('stopwords')

# Set stop words
stop_words = stopwords.words('english')

In [ ]:
# Tokenize and remove stopwords
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

In [ ]:
data = data['stack'].tolist()
data_words = list(sent_to_words(data))
print(data_words[:1][0][:30])

In [ ]:
# Create dictionary and corpus
id2word = corpora.Dictionary(data_words)
texts = data_words
corpus = [id2word.doc2bow(text) for text in texts]
print(corpus[:1][0][:30])

In [ ]:
# Function to train LDA model and compute coherence score
def train_and_compute_coherence(corpus, dictionary, texts, num_topics, random_state=0):
    lda_model = LdaMulticore(corpus=corpus, id2word=dictionary, num_topics=num_topics, 
                             random_state=random_state, chunksize=100, passes=10, alpha=0.01, eta=0.9)
    coherence_model = CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    return lda_model, coherence_score


In [ ]:
# Determine the best number of topics
num_topics_range = range(2, 11)
coherence_scores = []
random_state = 0


In [ ]:
# model saving path
model_path_template = 'lda_multicore_model_{}.model'
ldavis_path_template = '/mnt/home/kim/thesis/lda-figure/anes/ldavis_stack_{}.pkl'

## warning: don't run it, otherwise it will change the result

In [ ]:
for num_topics in num_topics_range:
    lda_model, coherence_score = train_and_compute_coherence(corpus, id2word, texts, num_topics, random_state)
    lda_model.save(model_path_template.format(num_topics))  # Saving the model
    coherence_scores.append((num_topics, coherence_score))

In [ ]:
# Print coherence scores
for num_topics, score in coherence_scores:
    print(f'Num Topics: {num_topics}, Coherence Score: {score}')

In [ ]:
# Find the best number of topics
best_num_topics, best_coherence_score = max(coherence_scores, key=lambda item: item[1])
print(f'Best Num Topics: {best_num_topics}, Best Coherence Score: {best_coherence_score}')

### reload the model that is saved. 

In [ ]:
# Reload the best model
best_model_path = model_path_template.format(best_num_topics)
lda_model = LdaMulticore.load(best_model_path)

In [ ]:
# Visualize using PyLDAvis
pyLDAvis.enable_notebook()
LDAvis_data_filepath = ldavis_path_template.format(best_num_topics)

if not os.path.exists(LDAvis_data_filepath):
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
else:
    with open(LDAvis_data_filepath, 'rb') as f:
        LDAvis_prepared = pickle.load(f)

html_filepath = LDAvis_data_filepath.replace('.pkl', '.html')
pyLDAvis.save_html(LDAvis_prepared, html_filepath)


In [ ]:
# Display the visualization inline
LDAvis_prepared

### 3. Dataset: responses stacked
### * with tf-idf

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Load data
data = pd.read_json('~/thesis/data/processed_uscensus/political_mentions_stack.jsonl', orient='records', lines=True)

In [ ]:
nltk.download('stopwords')

# Set stop words
stop_words = stopwords.words('english')

In [ ]:
# Tokenize and remove stopwords
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

data = data['stack'].tolist()
data_words = list(sent_to_words(data))
print(data_words[:1][0][:30])

In [ ]:
# Flatten the tokenized words back into strings
data_words_strings = [' '.join(words) for words in data_words]

In [ ]:
# Initialize the TF-IDF Vectorizer
vectorizer = TfidfVectorizer(max_features=1000)

# Fit the vectorizer and transform the documents
X_tfidf = vectorizer.fit_transform(data_words_strings)

# Get the feature names (i.e., words) and create a mapping for Gensim
feature_names = vectorizer.get_feature_names_out()
id2word = Dictionary([feature_names.tolist()])


In [ ]:
# Convert the TF-IDF matrix to a Gensim-compatible corpus format
corpus = []
for doc in X_tfidf:
    doc_tuples = list(enumerate(doc.toarray()[0]))
    doc_tuples = [(i, val) for i, val in doc_tuples if val > 0]
    corpus.append(doc_tuples)

In [ ]:
# View the first document's first 30 tokens
print(corpus[:1][0][:30])

In [ ]:
# Function to train LDA model and compute coherence score
def train_and_compute_coherence(corpus, dictionary, texts, num_topics, random_state=0):
    lda_model = LdaMulticore(corpus=corpus, id2word=dictionary, num_topics=num_topics, 
                             random_state=random_state, chunksize=100, passes=10, alpha=0.01, eta=0.9)
    coherence_model = CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    return lda_model, coherence_score


In [ ]:
# Determine the best number of topics
num_topics_range = range(2, 11)
coherence_scores = []
random_state = 0


### warning: don't run it, otherwise it will rewrite. 

In [ ]:
# save the path and reload it later
model_path_template = 'lda_multicore_model_{}.model'
ldavis_path_template = '/mnt/home/kim/thesis/lda-figure/anes/ldavis_{}.pkl'

In [ ]:
for num_topics in num_topics_range:
    lda_model, coherence_score = train_and_compute_coherence(corpus, id2word, data_words, num_topics, random_state)
    lda_model.save(model_path_template.format(num_topics))
    coherence_scores.append((num_topics, coherence_score))


In [ ]:
# Print coherence scores
for num_topics, score in coherence_scores:
    print(f'Num Topics: {num_topics}, Coherence Score: {score}')


In [ ]:
# Find the best number of topics
best_num_topics, best_coherence_score = max(coherence_scores, key=lambda item: item[1])
print(f'Best Num Topics: {best_num_topics}, Best Coherence Score: {best_coherence_score}')


### reload the LDA model from here

In [ ]:
# Reload the best model
best_model_path = model_path_template.format(best_num_topics)
lda_model = LdaMulticore.load(best_model_path)

In [ ]:
# Visualize using PyLDAvis
pyLDAvis.enable_notebook()
LDAvis_data_filepath = ldavis_path_template.format(best_num_topics)

if not os.path.exists(LDAvis_data_filepath):
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
else:
    with open(LDAvis_data_filepath, 'rb') as f:
        LDAvis_prepared = pickle.load(f)

html_filepath = LDAvis_data_filepath.replace('.pkl', '.html')
pyLDAvis.save_html(LDAvis_prepared, html_filepath)

In [ ]:
# Display the visualization inline
LDAvis_prepared

### 3. Dataset: responses stacked
### * with tf-idf

In [ ]:
data = pd.read_json('~/thesis/data/processed_uscensus/political_mentions_stack.jsonl', orient='records', lines = True)

In [ ]:
data.head()

In [ ]:
# Tokenising and removing the stopwords
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]

In [ ]:
data = data['stack'].tolist()
data_words = list(sent_to_words(data))

# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

In [ ]:
data_words[:1]

In [ ]:
# Step 1: Flatten the tokenized words back into strings
data_words_strings = [' '.join(words) for words in data_words]


In [ ]:
# Step 2: Initialize the TF-IDF Vectorizer
vectorizer = TfidfVectorizer(max_features=1000)

# Step 3: Fit the vectorizer and transform the documents
X_tfidf = vectorizer.fit_transform(data_words_strings)

In [ ]:
# Step 4: Get the feature names (i.e., words) and create a mapping for Gensim
feature_names = vectorizer.get_feature_names_out()
id2word = gensim.corpora.Dictionary([feature_names])

# Step 5: Convert the TF-IDF matrix to a Gensim-compatible corpus format
corpus = []
for doc in X_tfidf:
    doc_tuples = list(enumerate(doc.toarray()[0]))
    doc_tuples = [(i, val) for i, val in doc_tuples if val > 0]
    corpus.append(doc_tuples)

# View the first document's first 30 tokens
print(corpus[:1][0][:30])


In [ ]:
# Step 6: Build the LDA model using the TF-IDF-based corpus
num_topics = 6  # number of topics based on the clustering result from the previous analysis

lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics,
                                       random_state=0,
                                       chunksize=100,
                                       passes=10,
                                       alpha=0.01,
                                       eta=0.9)

In [ ]:
from pprint import pprint

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

### Visualize the result of topic modelling

In [ ]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis

# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('/mnt/home/kim/thesis/lda-figure/anes/ldavis_'+str(num_topics))

In [ ]:
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, '/mnt/home/kim/thesis/lda-figure/anes/ldavis_'+ str(num_topics) +'.html')
LDAvis_prepared

## Additionally: 

### Top words

In [ ]:
from gensim.utils import simple_preprocess
from collections import Counter
from itertools import combinations
import pickle

In [ ]:
# Extract top words for each topic
top_words_per_topic = []
for t in range(num_topics):
    top_words = [word for word, _ in lda_model.show_topic(t, topn=10)]
    top_words_per_topic.append(top_words)

In [ ]:
top_words_per_topic[:5]

In [ ]:
# Compute co-occurrence matrix
def compute_cooccurrence_matrix(texts):
    word_counts = Counter(word for text in texts for word in text)
    total_count = sum(word_counts.values())
    word_pairs = Counter()
    for text in texts:
        for i, j in combinations(set(text), 2):
            word_pairs[tuple(sorted([i, j]))] += 1
    return word_pairs, word_counts, total_count

word_pairs, word_counts, total_count = compute_cooccurrence_matrix(data_words)

# Compute NPMI
def compute_npmi(word_pairs, word_counts, total_count):
    npmi_matrix = {}
    for (w_i, w_j), cooccur_count in word_pairs.items():
        p_i = word_counts[w_i] / total_count
        p_j = word_counts[w_j] / total_count
        p_ij = cooccur_count / total_count
        if p_ij > 0:
            pmi = np.log(p_ij / (p_i * p_j))
            npmi = pmi / -np.log(p_ij)
            npmi_matrix[(w_i, w_j)] = npmi
    return npmi_matrix

npmi_matrix = compute_npmi(word_pairs, word_counts, total_count)

In [ ]:
# Calculate average NPMI for each topic
def average_npmi_for_topics(top_words_per_topic, npmi_matrix):
    topic_npmis = []
    for top_words in top_words_per_topic:
        npmis = [npmi_matrix.get(tuple(sorted([w_i, w_j])), 0) for w_i, w_j in combinations(top_words, 2)]
        if npmis:
            topic_npmi = np.mean(npmis)
            topic_npmis.append(topic_npmi)
    return np.mean(topic_npmis) if topic_npmis else 0

average_npmi = average_npmi_for_topics(top_words_per_topic, npmi_matrix)
print("Average NPMI for LDA topics:", average_npmi)

**Interpretation**

High NPMI (close to 1): Indicates strong semantic coherence between words, meaning the words are likely to appear together in similar contexts. This is generally considered good for topics generated by models like LDA.

NPMI around 0: Indicates that the words appear together about as frequently as expected by chance, suggesting neutral association.

Low NPMI (negative values): Indicates that the words are unlikely to appear together, suggesting poor coherence for the topic.